In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.sql.functions._

val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

def sc = spark.sparkContext
import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark


import org.apache.spark.sql._


import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = 

In [2]:
val df = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("data.csv")

df: DataFrame = [device_id: string, country_code: string ... 12 more fields]

In [46]:
def funnel(dataframe_funnel: DataFrame, arr_funnel_events:Array[String]):Unit = {
    var arr = Array("id") ++ arr_funnel_events                                                                  // this array contains "id" and funnel events
    var df = dataframe_funnel.select(col("device_id"), col("timestamp"), col("event_type"))                     // just filtering the columns we need from dataframe
    var fun_df = df.groupBy("device_id").agg(min("timestamp").as("time")).select('device_id.as("id_0"), 'time)  // we get unique ids and the minimum timestamp for each id
    val byDeviceId = Window.partitionBy(col("id_0")).orderBy('time.asc)                                         // this window function is used to keep only first occurence of funnel event. we later use rank for that purpose
    for (x <- 1 until arr.length){  
        fun_df = fun_df.
        join(df, col(arr(x-1).concat("_"+(x-1).toString)).isNotNull && col("id_0") === 'device_id && 'event_type === arr(x) && 'time <= 'timestamp, "left").
        drop("time", "device_id").
        withColumnRenamed("event_type", arr(x).concat("_"+x.toString)).
        withColumnRenamed("timestamp", "time").                                                                 // we drop old time and replace it with the timestamp of new event  
        withColumn("rank", rank over byDeviceId).where('rank === 1).                                            // we keep only the first occurence based on timestamp for each id
        drop('rank) 
        }
    fun_df = fun_df.drop("time")
    fun_df.distinct.select(fun_df.columns.map(c => count(col(c)).alias(c)): _*).show()                          // show count of each column
}

var arr = Array("app_open", "editor_done_click", "edit_sticker_category_open", "editor_done_click")
funnel(df, arr)

+----+----------+-------------------+----------------------------+-------------------+
|id_0|app_open_1|editor_done_click_2|edit_sticker_category_open_3|editor_done_click_4|
+----+----------+-------------------+----------------------------+-------------------+
|  98|        89|                 69|                          22|                 20|
+----+----------+-------------------+----------------------------+-------------------+



defined function funnel
arr: Array[String] = Array(
  "app_open",
  "editor_done_click",
  "edit_sticker_category_open",
  "editor_done_click"
)